In [1]:
import pandas as pd

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
annotation_df = pd.read_csv('coco_annotations_socnosoc.csv')

In [4]:
nsd_stim_df = pd.read_csv('nsd_stim_info_merged.csv')

In [5]:
#Will need to add in all participant files here:
behav_files = ['../../../shared/NSD/nsddata/ppdata/subj01/behav/responses.tsv',
              '../../../shared/NSD/nsddata/ppdata/subj02/behav/responses.tsv',
              '../../../shared/NSD/nsddata/ppdata/subj03/behav/responses.tsv',
              '../../../shared/NSD/nsddata/ppdata/subj04/behav/responses.tsv',
              '../../../shared/NSD/nsddata/ppdata/subj05/behav/responses.tsv',
              '../../../shared/NSD/nsddata/ppdata/subj06/behav/responses.tsv',
              '../../../shared/NSD/nsddata/ppdata/subj07/behav/responses.tsv',
              '../../../shared/NSD/nsddata/ppdata/subj08/behav/responses.tsv',
              ]
first_file = True
for sub_tsv in behav_files:
    # curr_df = pd.read_csv(sub_tsv, sep='\t')
    # print(len(curr_df))
    # print(curr_df['SESSION'].nunique())
    if first_file != True:
        add_df = pd.read_csv(sub_tsv, sep='\t')
        behavioral_info_df = pd.concat([behavioral_info_df, add_df])
    else:
        behavioral_info_df = pd.read_csv(sub_tsv, sep='\t')
        first_file = False

In [6]:
behavioral_info_df['73KID_0idx'] = behavioral_info_df['73KID'] - 1

In [7]:
nsd_ann_total = nsd_stim_df.merge(annotation_df, how='left', left_on='cocoId', right_on='id')
nsd_ann_total.drop('id', axis=1, inplace=True)

In [8]:
behavioral_info_df['trial_in_session'] = behavioral_info_df.groupby(['SUBJECT', 
                                                                     'SESSION']).cumcount()

In [9]:
behav_ann_comb_df = behavioral_info_df.merge(nsd_ann_total, 
                    how='left', 
                    left_on='73KID_0idx',
                    right_on='nsdId')[['SUBJECT', 'SESSION', 'RUN', 'TRIAL', 
                                       'trial_in_session', 'cocoId', 'nsdId', 
                                       'flagged', 'BOLD5000', 'shared1000', 'file_name', 
                                       'coco_url','people_coco', 'people', 'faces', 
                                       'people_coco_count', 'people_count', 'faces_count',
                                       '73KID', '10KID', 'TIME', 'ISOLD', 'ISCORRECT', 
                                       'RT', 'CHANGEMIND', 'MEMORYRECENT', 'MEMORYFIRST',
                                       'ISOLDCURRENT', 'ISCORRECTCURRENT', 'TOTAL1', 
                                       'TOTAL2', 'BUTTON', 'MISSINGDATA', '73KID_0idx']]

In [10]:
hand_ann_df = pd.read_csv('interrater_reliability.csv')[['nsdID', 'Social/Nonsocial']]

In [11]:
hand_ann_dict = hand_ann_df.set_index('nsdID').to_dict()['Social/Nonsocial']

In [12]:
def apply_socnonsoc(nsdid):
    if nsdid in hand_ann_dict.keys():
        if hand_ann_dict[nsdid] == 'social':
            return 1
        elif hand_ann_dict[nsdid] == 'nonsocial':
            return 0
    else:
        return float('nan')

In [13]:
behav_ann_comb_df['human'] = behav_ann_comb_df['nsdId'].apply(apply_socnonsoc)

In [14]:
# behav_ann_comb_df[behav_ann_comb_df['shared1000'] == True][['SUBJECT', 'SESSION',
#                                                             'RUN', 'trial_in_session'
#                                                            ]].to_csv('shared_trial_info.csv')

In [15]:
# behav_ann_comb_df.to_csv('annotations_behavioral_combined.csv')

In [16]:
behav_ann_comb_df.rename(columns={'people_coco': 'coco', 'people': 'yolo'}, inplace=True)

In [17]:
ann_df = behav_ann_comb_df[behav_ann_comb_df['shared1000'] == True][['nsdId', 'human', 
                                                            'coco', 'yolo']]

In [18]:
ann_df = ann_df.drop_duplicates()

In [19]:
# ann_df.to_csv('annotations.csv')

In [20]:
final_ann = pd.read_csv('annotation_decisions.csv').set_index('nsdId')

final_ann.rename(columns={'final decision':'final_decision'}, inplace=True)

In [21]:
final_ann_dict = final_ann[['final_decision']].to_dict()['final_decision']

In [22]:
behav_ann_comb_df['final_socnonsoc'] = behav_ann_comb_df['nsdId'].map(final_ann_dict)

In [23]:
behav_ann_comb_df.drop( ['human', 'yolo', 'faces', 'coco'], axis=1, inplace=True)

In [24]:
df_data = pd.read_csv('../data_processing/betas_sub1_ses1.csv').rename(columns={
                                                                    'Trial':'trial_in_session',
                                                                    })

In [25]:
behav_ann_comb_df = behav_ann_comb_df[behav_ann_comb_df['shared1000'] == True]

In [32]:
behav_ann_comb_df['final_socnonsoc'].value_counts(dropna=False)

0.0    11972
1.0     9130
NaN       16
Name: final_socnonsoc, dtype: int64

In [35]:
#Change to not be hard coded if we share this code.
# behav_ann_comb_df[behav_ann_comb_df['final_socnonsoc'].isna()]
behav_ann_comb_df['final_socnonsoc'] = behav_ann_comb_df['final_socnonsoc'].fillna(0)

In [36]:
behav_ann_comb_df.merge(df_data, 
                        how='left', 
                        on='trial_in_session')[['trial_in_session',
                                                'final_socnonsoc']].to_csv(
                                                                        'sample_input_output.csv'
                                                                        )